# Biogeme Basic Logit Model

In [10]:
import pandas  as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.optimization as opt
from biogeme.expressions import Beta, DefineVariable
import biogeme.expressions as exp
import seaborn as sns
import matplotlib.pyplot as plt

## import data 
using swiss_metro data as example

In [3]:
swissmetro = pd.read_csv("/Users/lei/Documents/毕业论文/biogeme_logit_learning/swissmetro.dat",sep='\t')
database = db.Database("swissmetro", swissmetro)

## 数据说明

### 调查简介

* 这个数据是一个SP调查，在一趟火车上收集的。
* 这个数据描述了人们对一种新的交通方式“真空列车”的看法，这种新方式与公共交通和小汽车不同。SA公司提供这项服务，在部分真空状态下能够达到500公里的运行速度。
* 两组调查：
    * 一组来自于铁路用户。最终441个参与者完成了调查，每个受访者生成了9种sp场景，三种方式选择：铁路，真空列车，汽车（仅针对汽车所有者）。
    * 一组来源于小汽车用户。基于车牌的调查，是从高速公路上拿到的车牌所有者，然后，给这些人寄问卷。填写了rp调查，然后参加了sp调查。共750份可用。

In [4]:
swissmetro.columns

Index(['GROUP', 'SURVEY', 'SP', 'ID', 'PURPOSE', 'FIRST', 'TICKET', 'WHO',
       'LUGGAGE', 'AGE', 'MALE', 'INCOME', 'GA', 'ORIGIN', 'DEST', 'TRAIN_AV',
       'CAR_AV', 'SM_AV', 'TRAIN_TT', 'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO',
       'SM_HE', 'SM_SEATS', 'CAR_TT', 'CAR_CO', 'CHOICE'],
      dtype='object')

### 字段说明

GROUP：问卷来源。2 铁路用户，3 小汽车用户。
SURVEY：调查组别，与GROUP相同。0 铁路用户，1 汽车用户。
SP：1 sp调查
ID：受访者唯一标识
PURPOSE：出行目的。1 通勤；2 购物；3 商务；4 休闲；
                 5 下班返回；6 购物返回；7 商务返回；8 休闲返回；
                 9 其他。
FIRST：是否是头等舱用户。0 否，1 是。
TICKET：铁路卡种类。0 无； 1 半价卡往返； 2 单程半价卡；3 正价往返票；4 单程正价卡；
5 半天票；6 季票；7 高级季票；8 晚7点之后免费卡；9 团体票； 10 其他。
WHO：谁支付价格。0 未知； 1 自己； 2 雇主； 3 一半一半。
LUGGAGE： 有无行李。0 无；1 一件；2 大于1件。
AGE： 年龄。分组年龄 1 24岁及以下；2 24-39岁；3 39-54岁；4 54-65岁；5 大于65；6 未知
MALE：性别。0 女；1 男。
INCOME： 收入。分组收入。
GA: 是否有年票铁路卡。1 有；0 无。
ORIGIN：旅行出发地。每个区域对应一个数字。
DEST： 旅行目的地。每个区域对应一个数字。
TRAIN_AV：列车可用性。
CAR_AV：小汽车可用性。
SM_AV：真空列车可用性。
TRAIN_TT：列车出行时长，分钟。是距离的函数。
TRAIN_CO：列车出行费用，瑞法。如果有GA，这个费用是年票的价格。
TRAIN_HE：列车发车频率，分钟。如果每小时有两列火车，则该值为30。
SM_TT：真空列车出行时长，分钟。是距离的函数，每小时500公里。
SM_CO：真空列车的费用，瑞法。固定系数计算。
SM_HE：真空列车的发车频率。
SM_SEATS：是否有座位。1 有；0 无。
CAR_TT：小汽车出行时长，分钟。是距离的函数，每小时500公里。
CAR_CO：小汽车的费用，瑞法。固定系数计算。
CHOICE：选择了啥。0: 未知；1 列车； 2 真空列车；3 小汽车

In [5]:
swissmetro

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,3,1,1,1192,4,1,7,1,0,5,...,148,13,30,93,17,30,0,156,56,2
10724,3,1,1,1192,4,1,7,1,0,5,...,148,12,30,96,16,10,0,96,70,3
10725,3,1,1,1192,4,1,7,1,0,5,...,148,16,60,93,16,20,0,96,56,3
10726,3,1,1,1192,4,1,7,1,0,5,...,178,16,30,96,17,30,0,96,91,2


## 模型配置

变量名作为全局变量名

In [7]:
globals().update(database.variables)

筛选数据

条件是 目的不为1，或者，目的不为3，并且选择 等于0
仅保留了通勤的人。

In [8]:
exclude = (( PURPOSE != 1 ) * ( PURPOSE != 3 ) + ( CHOICE == 0 )) > 0

database.remove(exclude)

定义虚拟变量以及处理一些变量。


In [12]:
SM_COST = SM_CO * ( GA == 0 ) # 如果没有GA卡，SM_cost 为0
TRAIN_COST = TRAIN_CO * ( GA == 0 ) # 如果没有GA卡，TRAIN_COST 为0

CAR_AV_SP = database.DefineVariable ('CAR_AV_SP', CAR_AV * ( SP !=0 ))
# 小汽车可用性变量。如果SP不等于0，小汽车可用性不变，否则为0
TRAIN_AV_SP = database.DefineVariable ('TRAIN_AV_SP', TRAIN_AV * ( SP != 0 ))
# 列车可用性变量。如果SP不等于0，列车可用性不变，否则为0

一些数据的值进行缩放

In [13]:
TRAIN_TT_SCALED   = database.DefineVariable('TRAIN_TT_SCALED',   TRAIN_TT / 100.0)
TRAIN_COST_SCALED = database.DefineVariable('TRAIN_COST_SCALED', TRAIN_COST / 100)
SM_TT_SCALED      = database.DefineVariable('SM_TT_SCALED',      SM_TT / 100.0)
SM_COST_SCALED    = database.DefineVariable('SM_COST_SCALED',    SM_COST / 100)
CAR_TT_SCALED     = database.DefineVariable('CAR_TT_SCALED',     CAR_TT / 100)
CAR_CO_SCALED     = database.DefineVariable('CAR_CO_SCALED',     CAR_CO / 100 )

查看修改之后的数据

In [14]:
swissmetro_after = database.data

In [15]:
swissmetro_after

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,CAR_CO,CHOICE,CAR_AV_SP,TRAIN_AV_SP,TRAIN_TT_SCALED,TRAIN_COST_SCALED,SM_TT_SCALED,SM_COST_SCALED,CAR_TT_SCALED,CAR_CO_SCALED
0,2,0,1,1,1,0,1,1,0,3,...,65,2,1.0,1.0,1.12,0.48,0.63,0.52,1.17,0.65
1,2,0,1,1,1,0,1,1,0,3,...,84,2,1.0,1.0,1.03,0.48,0.60,0.49,1.17,0.84
2,2,0,1,1,1,0,1,1,0,3,...,52,2,1.0,1.0,1.30,0.48,0.67,0.58,1.17,0.52
3,2,0,1,1,1,0,1,1,0,3,...,52,2,1.0,1.0,1.03,0.40,0.63,0.52,0.72,0.52
4,2,0,1,1,1,0,1,1,0,3,...,84,2,1.0,1.0,1.30,0.36,0.63,0.42,0.90,0.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8446,3,1,1,939,3,1,7,3,1,5,...,64,1,1.0,1.0,1.08,0.13,0.50,0.17,1.30,0.64
8447,3,1,1,939,3,1,7,3,1,5,...,80,1,1.0,1.0,1.08,0.12,0.53,0.16,0.80,0.80
8448,3,1,1,939,3,1,7,3,1,5,...,64,1,1.0,1.0,1.08,0.16,0.50,0.16,0.80,0.64
8449,3,1,1,939,3,1,7,3,1,5,...,104,1,1.0,1.0,1.28,0.16,0.53,0.17,0.80,1.04


## 定义效用函数

   \begin{equation}
    V_1 & = \beta_{Train} + \beta_{time}X_{Train_{TT}} + \beta_{cost}X_{Train_{cost}} \\
    V_2 & = \beta_{SM} +  \beta_{time}X_{SM_{TT}} + \beta_{cost}X_{SM_{cost}} \\
    V_3 & = \beta_{Car} + \beta_{time}X_{Car_{TT}} + \beta_{cost}X_{Car_{cost}}
   \end{equation}

定义要标定的模型的参数

In [16]:
ASC_CAR   = exp.Beta('ASC_CAR',0,None ,None ,0)  # 常数，相当于`$\beta{car}$`
ASC_TRAIN = exp.Beta('ASC_TRAIN',0,None ,None ,0)
ASC_SM    = exp.Beta('ASC_SM',0,None ,None ,1)
B_TIME    = exp.Beta('B_TIME',0,None ,None ,0)  # 出行时长前面的系数
B_COST    = exp.Beta('B_COST',0,None ,None ,0)  # 出行费用前面的系数

形式化模型的效用函数

In [17]:
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED


把效用函数 和 方式可用性联合起来

注意，这里的顺序不能混乱掉。v中的1，表达了train的效用函数，av1 里边的1 就必须是train的可用性

In [18]:
V = {1: V1,
     2: V2,
     3: V3}

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}



## 初始化和定义模型

In [19]:
logprob = models.loglogit(V, av, CHOICE) # 用log likelihood 来标定模型
                          
biogeme  = bio.BIOGEME(database, logprob)
biogeme.modelName = "swissmetro_logit_basic"

File biogeme.toml has been created


## 拟合模型

In [20]:
biogeme.generateHtml = True
biogeme.generatePickle = False

results = biogeme.estimate()

print(f"HTML file:    {results.data.htmlFileName}")
print(f"Pickle file:  {results.data.pickleFileName }")

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


HTML file:    swissmetro_logit_basic.html
Pickle file:  None


## 模型最终输出

In [21]:
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k:10}=\t{v:.3g}")

ASC_CAR   =	-0.155
ASC_TRAIN =	-0.701
B_COST    =	-1.08
B_TIME    =	-1.28
